# Part 1

## These are just preliminary tests for now (08/10/2021 5pm)

In [3]:
# Importing standard modules
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from typing import List

import logisticregression as logreg

NameError: name 'logisticregression' is not defined

In [ ]:
train = pd.read_csv('../data/diabetes/diabetes_train.csv')
test = pd.read_csv('../data/diabetes/diabetes_test.csv')
valid = pd.read_csv('../data/diabetes/diabetes_val.csv')

train.head(10)

In [ ]:
frames = [train, test, valid]
all_data = pd.concat(frames, axis=0)

all_data.head(10)

In [ ]:
# Function to find convergent solution of gradient descent as a function of learning-rate and maximum iterations

def optimize(max_iter, learning_rate, data):
    
    # Input data, features and binary labels column
    Xin=data.iloc[:,1:]
    Yin=data.iloc[:,0]
    
    # Split data into training and validation data sets
    X_train, X_valid, Y_train, Y_valid = train_test_split(Xin, Yin, test_size=0.2, random_state=0)
    
    # Iterate through the input parameters
    for l in learning_rate:
        print("Learning Rate: ", l)
        for m in max_iter:
            print("Maximum Iterations: ", m)
            model = logreg.LogisticRegression(verbose=True, add_bias=True, learning_rate=l, max_iters=m)
            yh = model.fit(X_train,Y_train).predict(X_valid)


mi = [1e4,9e4,1e5]
lr = [.1,1,10,0.001]

optimize(max_iter=mi, learning_rate=lr, data=all_data)
    